In [1]:
from fh_id_data import fh_id_data
from fh_ca import campo_amostral
from fh_filter import get_filter
import os
from supabase import create_client, Client
import math
table = 'table_statics'
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)


In [ ]:
def mount_gols_filters(fh_q_dict, dict_campo_amostral):
    category = 'gols'
    gol_atual = fh_q_dict['gols']['sum_period1']
    ca_category = dict_campo_amostral['gols'] 
    print(gol_atual)
    dict_gols_filter_over = {
        1 :  [gol_atual + 0.5, get_filter(fh_q_dict, category, 1),0,0],
        2 : [gol_atual + 1.5, get_filter(fh_q_dict, category, 2), 0,0],
        3 : [gol_atual + 2.5, get_filter(fh_q_dict, category, 3), 0,0]
    }

    dict_gols_filter_under = {
        1 : [gol_atual + 0.5 ,ca_category - get_filter(fh_q_dict, category, 1), 0, 0],
        2 : [gol_atual + 1.5, ca_category - get_filter(fh_q_dict, category, 2),0, 0],
        3 : [gol_atual + 2.5, ca_category - get_filter(fh_q_dict, category, 3),0, 0]
    }

    return dict_gols_filter_over, dict_gols_filter_under


def mount_gols_resp(dict_gols_filter_over, dict_gols_filter_under):
    for i in range(1,len(dict_gols_filter_over) + 1):
        prob = (dict_gols_filter_over[i][1] * 100) / dict_campo_amostral['gols']
        dict_gols_filter_over[i][2] = prob
        odd = 1/(prob/100)
        dict_gols_filter_over[i][3] = odd

    for i in range(1,len(dict_gols_filter_under) + 1):
        prob = (dict_gols_filter_under[i][1] * 100) / dict_campo_amostral['gols']
        dict_gols_filter_under[i][2] = prob
        odd = 1/(prob/100)
        dict_gols_filter_under[i][3] = odd

    return dict_gols_filter_over, dict_gols_filter_under




In [57]:
def mount_corners_filters(fh_q_dict, dict_campo_amostral):
    category = 'corners'
    corners_atual = fh_q_dict['corners']['sum_1st_cornerkicks'][2]
    ca_category = dict_campo_amostral['corners'] 
    print(corners_atual)
    dict_corners_filter_over = {
        1 : [corners_atual + 3, get_filter(fh_q_dict, category, 3),0,0],
        2 : [corners_atual + 4, get_filter(fh_q_dict, category, 4), 0,0],
        3 : [corners_atual + 5, get_filter(fh_q_dict, category, 5), 0,0],
        4 : [corners_atual + 6, get_filter(fh_q_dict, category, 6), 0,0],
        5 : [corners_atual + 7, get_filter(fh_q_dict, category, 7), 0,0],
    }

    dict_corners_filter_under = {
        1 : [corners_atual + 3, ca_category - get_filter(fh_q_dict, category, 3),0, 0],
        2 : [corners_atual + 4, ca_category - get_filter(fh_q_dict, category, 4),0, 0],
        3 : [corners_atual + 5, ca_category - get_filter(fh_q_dict, category, 5),0, 0],
        4 : [corners_atual + 6, ca_category - get_filter(fh_q_dict, category, 6),0, 0],
        5 : [corners_atual + 7, ca_category - get_filter(fh_q_dict, category, 7),0, 0],
    }

    return dict_corners_filter_over, dict_corners_filter_under



def mount_corners_resp(dict_corners_filter_over, dict_corners_filter_under):
    for i in range(1,len(dict_corners_filter_over) + 1):
        prob = (dict_corners_filter_over[i][1] * 100) / dict_campo_amostral['gols']
        dict_corners_filter_over[i][2] = prob
        odd = 1/(prob/100)
        dict_corners_filter_over[i][3] = odd

    for i in range(1,len(dict_corners_filter_under) + 1):
        prob = (dict_corners_filter_under[i][1] * 100) / dict_campo_amostral['gols']
        dict_corners_filter_under[i][2] = prob
        if prob != 0:
            odd = 1/(prob/100)
        elif prob == 0:
            odd = 10
        dict_corners_filter_under[i][3] = odd

    return dict_corners_filter_over, dict_corners_filter_under


In [62]:
event_id = 13472617
dict_campo_amostral, fh_q_dict = campo_amostral(event_id)
dict_campo_amostral, fh_q_dict

({'gols': 148, 'corners': 30, 'yellowcards': 508},
 {'gols': {'sum_period1': 1,
   'sum_expected_goals_1st': (1.8200000000000003, 3.8200000000000003),
   'sum_1st_totalshotsongoal': (15, 25),
   'sum_1st_shotsongoal': (0, 10),
   'sum_1st_totalshotsinsidebox': (8, 18),
   'sum_1st_finalthirdentries': (-9, 91),
   'sum_1st_cornerkicks': (-2, 8, 3),
   'sum_1st_freekicks': (8, 18),
   'sum_1st_yellowcards': (-3, 5, 1)},
  'corners': {'sum_period1': 1,
   'sum_expected_goals_1st': (1.8200000000000003, 3.8200000000000003),
   'sum_1st_totalshotsongoal': (15, 25),
   'sum_1st_shotsongoal': (0, 10),
   'sum_1st_totalshotsinsidebox': (8, 18),
   'sum_1st_finalthirdentries': (-9, 91),
   'sum_1st_cornerkicks': (-2, 8, 3),
   'sum_1st_freekicks': (8, 18),
   'sum_1st_yellowcards': (-3, 5, 1)},
  'yellowcards': {'sum_period1': 1,
   'sum_expected_goals_1st': (2.5700000000000003, 3.0700000000000003),
   'sum_1st_totalshotsongoal': (18, 22),
   'sum_1st_shotsongoal': (4, 6),
   'sum_1st_totalshots

In [63]:
dict_corners_filter_over, dict_corners_filter_under = mount_corners_filters(fh_q_dict, dict_campo_amostral)

display(dict_corners_filter_over)
display(dict_corners_filter_under)

3


{1: [6, 29, 0, 0],
 2: [7, 28, 0, 0],
 3: [8, 21, 0, 0],
 4: [9, 16, 0, 0],
 5: [10, 6, 0, 0]}

{1: [6, 1, 0, 0],
 2: [7, 2, 0, 0],
 3: [8, 9, 0, 0],
 4: [9, 14, 0, 0],
 5: [10, 24, 0, 0]}

In [64]:
mount_corners_resp(dict_corners_filter_over, dict_corners_filter_under)

({1: [6, 29, 19.594594594594593, 5.103448275862069],
  2: [7, 28, 18.91891891891892, 5.285714285714286],
  3: [8, 21, 14.18918918918919, 7.047619047619048],
  4: [9, 16, 10.81081081081081, 9.25],
  5: [10, 6, 4.054054054054054, 24.666666666666664]},
 {1: [6, 1, 0.6756756756756757, 148.0],
  2: [7, 2, 1.3513513513513513, 74.0],
  3: [8, 9, 6.081081081081081, 16.444444444444446],
  4: [9, 14, 9.45945945945946, 10.571428571428571],
  5: [10, 24, 16.216216216216218, 6.166666666666666]})

In [65]:
dict_gols_filter_over, dict_gols_filter_under = mount_gols_filters(fh_q_dict, dict_campo_amostral)
mount_gols_resp(dict_gols_filter_over, dict_gols_filter_under)

1


({1: [1.5, 130, 87.83783783783784, 1.1384615384615384],
  2: [2.5, 83, 56.08108108108108, 1.783132530120482],
  3: [3.5, 37, 25.0, 4.0]},
 {1: [1.5, 18, 12.162162162162161, 8.222222222222223],
  2: [2.5, 65, 43.91891891891892, 2.276923076923077],
  3: [3.5, 111, 75.0, 1.3333333333333333]})